In [2]:
from bs4 import BeautifulSoup, Comment
from urllib.request import urlopen
import pandas as pd
from selenium import webdriver
import time
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from random import *
from datetime import timedelta, date

# 연합뉴스 크롤링

In [6]:
raw_data = {'title':[],'date':[],'url':[],'script':[]}
rr = randrange(3)

# start, end 날짜 정의
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)


# 실제 코드에서는 2005.01.01 ~ 2017.12.31 수집
start_date = date(2017, 11, 1)
end_date = date(2017, 11, 2)          


# 크롤링        
for day in daterange(start_date, end_date):
    try:
        
        df = pd.DataFrame(raw_data)

        driver = webdriver.Chrome("chromedriver.exe")
        driver.get("https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EA%B8%88%EB%A6%AC&nso=so%3Add%2Cp%3Afrom" + day.strftime("%Y%m%d") + "to" + day.strftime("%Y%m%d") + "%2Ca%3Aall0")
        driver.set_page_load_timeout(30)

        driver.find_element_by_xpath("""//*[@id="news_popup"]/a""").click()
        time.sleep(rr)

        driver.find_element_by_xpath("""//*[@id="ca_1001"]""").click()
        time.sleep(rr)

        driver.find_element_by_xpath("""//*[@id="_nx_option_media"]/div[2]/div[3]/button[1]/span""").click()
        time.sleep(rr)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        a_tags = soup.find_all('a', class_='_sp_each_title')

        tmp = []
        tmp.append(a_tags)


        page = driver.find_elements_by_xpath("""//div[@class="paging"]//a""")
        time.sleep(rr)

        for i in range(1, len(page)):

            driver.get("https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EA%B8%88%EB%A6%AC&nso=so%3Add%2Cp%3Afrom" + day.strftime("%Y%m%d") + "to" + day.strftime("%Y%m%d") + "a:all&mynews=1&cluster_rank=60&start=" + str(i*10+1) + "&refresh_start=0")
            driver.set_page_load_timeout(30)



            soup = BeautifulSoup(driver.page_source, 'html.parser')
            a_tags = soup.find_all('a', class_='_sp_each_title')
            tmp.append(a_tags)
            time.sleep(rr)


        date=''
        prepro=''
        cnt= 1

        for k in range(len(tmp)):
            for j in tmp[k]:
            #제목
                print("제목 : " + j.get_text())
                title = j.get_text()
            #URL
                url = str(j['href'])
                driver.get(url)
                time.sleep(rr)
                soup = BeautifulSoup(driver.page_source, 'html.parser')

            #날짜
                if soup.find('div', class_='share-info') is not None:
                    date = soup.find('span', class_='tt').find('em')
                    date = str(date)
                    date = date[4:14]

            #본문
                if soup.find('div', class_='article') is not None:
                    div_tag = soup.find('div', class_='article')
                    prepro = str(div_tag)
                    prepro = re.sub('\n','',prepro,0).strip()
                    prepro = re.sub('<!--.+?-->','',prepro,0).strip()
                    prepro = re.sub('<.+?>','',prepro,0).strip()

                new_df = pd.DataFrame({'title':title, 'date':date, 'url':url, 'script':prepro}, index=[0])
                df = pd.concat([df, new_df])
            driver.close()
            driver = webdriver.Chrome("chromedriver.exe")

        df.to_csv("yeonhap" + day.strftime("%Y-%m-%d") + ".csv", header=True, index=False, encoding='cp949')

    except Exception:
        continue

제목 : 뉴욕증시, 연준 결정 앞두고 S&P·나스닥 사상 최고 출발
제목 : 금융硏 "올해 성장률 전망 3.1%로 상향…내년은 2.8%로 낮아져"(종합)
제목 : 경제 순항에 증시 활황…원/달러 환율, 3개월 만에 최저
제목 : 국고채 금리 일제히 하락…3년물 연 2.140%
제목 : 코스피 단숨에 33p '점프'…나흘째 사상 최고 '2,556.47'(종합2보)
제목 : 코스피 단숨에 33p '점프'…나흘째 사상최고 '2,556.47'(종합)
제목 : KIEP "美금리인상시 외자유출 가능성…금융시장 교란 없을듯"
제목 : 코스피 또 '점프'…4일째 최고치 마감 '2,556.47'(2보)
제목 : 주택보증공사, 가로주택정비사업 저리 기금융자 시작
제목 : 케이프투자 "비대면계좌 신규고객 주식수수료 평생 무료"
제목 : 금융硏 "올해 성장률 3.1%…내년은 2.8%"
제목 : [증시신상품] DB금융투자, 사명변경 기념 'DB 고배당 플러스랩'
제목 : 신한저축은행, 중금리 대출 금리 인하 이벤트
제목 : 10월 수출 '선방'…긴 추석연휴에도 7.1% 증가 449억달러(종합2보)
제목 : 전북교육청 금고 관리 기관에 농협은행 재선정
제목 : 질주하는 코스피 2,550선도 돌파…삼성전자 280만원 '훌쩍'(종합2보)
제목 : KEB하나은행, 평창올림픽 성공기원 예·적금 한정 판매
제목 : 코스피, 개장 직후 또 장중 최고치…2,540선도 넘어(종합)
제목 : 코스피, 개장 직후 또 장중 최고치…2,540선도 넘어(2보)
제목 : [전국 주요 신문 톱뉴스](1일 조간)
제목 : 뉴욕증시 연준 정책 결정 앞두고 강세…다우 0.12% 상승 마감
제목 : 국제유가, 감산연장 기대 상승지속…WTI 0.4%↑


# 연합인포맥스 크롤링

In [9]:
raw_data = {'title':[],'date':[],'url':[],'script':[]}
rr = randrange(3)


# start, end 날짜 정의    
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)


# 실제 코드에서는 2005.01.01 ~ 2017.12.31 수집
start_date = date(2017, 11, 1)
end_date = date(2017, 11, 2)           
    
    
# 크롤링     
for day in daterange(start_date, end_date):
    try:
        
        df = pd.DataFrame(raw_data)

        driver = webdriver.Chrome("chromedriver.exe")
        driver.get("https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EA%B8%88%EB%A6%AC&nso=so%3Add%2Cp%3Afrom" + day.strftime("%Y%m%d") + "to" + day.strftime("%Y%m%d") + "%2Ca%3Aall0")
        driver.set_page_load_timeout(30)

        driver.find_element_by_xpath("""//*[@id="news_popup"]/a""").click()
        time.sleep(rr)

        driver.find_element_by_xpath("""//*[@id="ca_2227"]""").click()
        time.sleep(rr)

        driver.find_element_by_xpath("""//*[@id="_nx_option_media"]/div[2]/div[3]/button[1]/span""").click()
        time.sleep(rr)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        a_tags = soup.find_all('a', class_='_sp_each_title')

        tmp = []
        tmp.append(a_tags)


        page = driver.find_elements_by_xpath("""//div[@class="paging"]//a""")
        time.sleep(rr)

        for i in range(1, len(page)):

            driver.get("https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EA%B8%88%EB%A6%AC&nso=so%3Add%2Cp%3Afrom" + day.strftime("%Y%m%d") + "to" + day.strftime("%Y%m%d") + "a:all&mynews=1&cluster_rank=60&start=" + str(i*10+1) + "&refresh_start=0")
            driver.set_page_load_timeout(30)



            soup = BeautifulSoup(driver.page_source, 'html.parser')
            a_tags = soup.find_all('a', class_='_sp_each_title')
            tmp.append(a_tags)
            time.sleep(rr)


        date=''
        prepro=''
        cnt= 1

        for k in range(len(tmp)):
            for j in tmp[k]:
                #제목
                print("제목 : " + j.get_text())
                title = j.get_text()
                #URL
                url = str(j['href'])
                driver.get(url)
                time.sleep(rr)
                soup = BeautifulSoup(driver.page_source, 'html.parser')

               #날짜
                if soup.find('div', class_='info-text') is not None:
                    date = soup.find('div', class_='info-text').find_all('li')
                    date = str(date[1].text)
                    date = date[4:14]

                #본문
                if soup.find_all('div', id='article-view-content-div') is not None:
                    div_tag = soup.find_all('div', id='article-view-content-div')
                    prepro = str(div_tag)
                    prepro = re.sub('\n','',prepro,0).strip()
                    prepro = re.sub('<!--.+?-->','',prepro,0).strip()
                    prepro = re.sub('<.+?>','',prepro,0).strip()

                new_df = pd.DataFrame({'title':title, 'date':date, 'url':url, 'script':prepro}, index=[0])
                df = pd.concat([df, new_df])

                driver.close()
                driver = webdriver.Chrome("chromedriver.exe")

            df.to_csv("infomax" + day.strftime("%Y-%m-%d") + ".txt", header=True, index=False)


    # 기사 삭제된 경우 - alert 클릭 후, 계속 진행
    except:
        print('ERROR')
        new_df = pd.DataFrame({'title':title, 'date':date, 'url':url, 'script':prepro}, index=[0])
        df = pd.concat([df, new_df])
        df.to_csv("infomax" + day.strftime("%Y-%m-%d") + ".txt", header=True, index=False)
        alert = driver.switch_to_alert()
        alert.accept()
        continue

제목 : 뉴욕증시, 연준 결정 앞두고 S&P·나스닥 사상 최고 출발
제목 : 미 국채가, FOMC 성명 앞두고 하락
제목 : SG, 순매도 우위 달러 추가 상승 여건 조성
제목 : BK 운용, 달러화 매파 FOMC 확인시 115엔까지 상승 전망
제목 : 영국 10월 제조업 PMI 56.3…예상치 상회(상보)
제목 : 금융위, 全 금융권 경쟁력 평가해 진입규제 낮춘다
제목 : <중국증시-마감> 경기 둔화 우려에 보합…0.08%↑(종합)
제목 : <채권-마감> 금리 하락…장 막판 단기물 매수 유입
제목 : IRS 금리 하락…커브 플래트닝
제목 : <서환-마감> 사상최고 코스피+외인 주식매수…5.90원↓
제목 : FX스와프 거침없는 상승세…CRS 금리 연동
제목 : SG 오석태 "중앙은행 펀치볼, 내년 최대 이슈"
제목 : 기재부 "韓-美 금리 역전, 외자유출ㆍ환율급등으로 연결 안 돼"
제목 : 가로주택정비사업 시동…인천 만수1 첫 주택기금 융자
제목 : <도쿄환시> 달러-엔 113엔대 후반 상승…FOMC 대기
제목 : KIEP "원화 내년에도 강세압력…금리 상승 가능성"
제목 : 한은, 국고채 단순매입 0.7조 전액 낙찰…응찰 2.55조
제목 : 달러-원 과매도 신호…연저점 경신은 '글쎄'
제목 : FT "차기 연준 의장, 경기 침체 준비해야…부양책 모색"
ERROR


C:\Users\user\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:101: DeprecationWarning: use driver.switch_to.alert instead


# 이데일리 크롤링

In [15]:
raw_data = {'title':[],'date':[],'url':[],'script':[]}
rr = randrange(3)
count = 0    


# start, end 날짜 정의    
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)


# 실제 코드에서는 2005.01.01 ~ 2017.12.31 수집
start_date = date(2017, 11, 1)
end_date = date(2017, 11, 2)         


# 크롤링 
for day in daterange(start_date, end_date):
    try:
        
        df = pd.DataFrame(raw_data)

        driver = webdriver.Chrome("chromedriver.exe")
        driver.get("https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EA%B8%88%EB%A6%AC&nso=so%3Add%2Cp%3Afrom" + day.strftime("%Y%m%d") + "to" + day.strftime("%Y%m%d") + "%2Ca%3Aall0")
        driver.set_page_load_timeout(30)

        driver.find_element_by_xpath("""//*[@id="news_popup"]/a""").click()
        time.sleep(rr)

        driver.find_element_by_xpath("""//*[@id="ca_1018"]""").click()
        time.sleep(rr)

        driver.find_element_by_xpath("""//*[@id="_nx_option_media"]/div[2]/div[3]/button[1]/span""").click()
        time.sleep(rr)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        a_tags = soup.find_all('a', class_='_sp_each_title')

        tmp = []
        tmp.append(a_tags)


        page = driver.find_elements_by_xpath("""//div[@class="paging"]//a""")
        time.sleep(rr)

        for i in range(1, len(page)):

            driver.get("https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EA%B8%88%EB%A6%AC&nso=so%3Add%2Cp%3Afrom" + day.strftime("%Y%m%d") + "to" + day.strftime("%Y%m%d") + "a:all&mynews=1&cluster_rank=60&start=" + str(i*10+1) + "&refresh_start=0")
            driver.set_page_load_timeout(30)



            soup = BeautifulSoup(driver.page_source, 'html.parser')
            a_tags = soup.find_all('a', class_='_sp_each_title')
            tmp.append(a_tags)
            time.sleep(rr)


        date=''
        prepro=''
        cnt= 1

        for k in range(len(tmp)):
            for j in tmp[k]:
            #제목
                print("제목 : " + j.get_text())
                title = j.get_text()
            #URL
                url = str(j['href'])
                driver.get(url)
                time.sleep(rr)
                soup = BeautifulSoup(driver.page_source, 'html.parser')

            #날짜
                if soup.find('div', class_='dates') is not None:
                    date = soup.find('div', class_='dates').find('p')
                    date = str(date)
                    date = date[6:16]

            #본문
                if soup.find('div', class_='news_body') is not None:
                    div_tag = soup.find('div', class_='news_body')
                    prepro = str(div_tag)
                    prepro = re.sub('\n','',prepro,0).strip()
                    prepro = re.sub('<!--.+?-->','',prepro,0).strip()
                    prepro = re.sub('<.+?>','',prepro,0).strip()

                new_df = pd.DataFrame({'title':title, 'date':date, 'url':url, 'script':prepro}, index=[0])
                df = pd.concat([df, new_df])
            driver.close()
            driver = webdriver.Chrome("chromedriver.exe")

#         df.to_csv("edaily" + day.strftime("%Y-%m-%d") + ".csv", header=True, index=False, encoding='utf-8')

    except Exception:
#         df.to_csv("edaily" + day.strftime("%Y-%m-%d") + ".csv", header=True, index=False, encoding='cp949')
        count = count + 1
        continue

제목 : 한국경제 잇단 '서프라이즈'…원화, 年 최고치 급등(종합)
제목 : [채권마감]장 막판 저가매수…롤러코스터 장세
제목 : '황금연휴 공포' 이겨낸 10월 수출… 3년만에 무역 1조달러 복귀할 듯(종합)
제목 : 한달새 3兆 사들인 외국인…원高에도 매물 걱정없는 4가지 이유
제목 : [마켓인]양극화에 때이른 북클로징…연말 회사채시장 `냉골`
제목 : [마켓인]1兆짜리 한남동 리스크…시장서 또 외면받은 대신F&I
제목 : 원·달러 환율 1110원대…연저점 가까워졌다
제목 : HUG, '인천만수'에 가로주택정비사업 기금융자 1호 지원
제목 : 케이프證, 주식·선물옵션 무료 수수료 행사…신용융자 이자율도↓
제목 : "5년내 경기침체기 온다…차기 연준 의장, 미리 대비해야"
제목 : 모바일증권 나무, 신규고객 국내주식 수수료 10년 무료 이벤트 실시
제목 : DB금융투자, 사명변경 기념 `DB 고배당 플러스 랩` 출시
제목 : KEB하나銀, ‘하나된 평창’ 전용 상품 출시… 입장권 이벤트 등
제목 : 대내외 불확실성 부담…채권 약보합
제목 : 코스피 2500돌파! 시장을 이끌 주도주는?
제목 : 신한저축銀, ‘국무총리 표창’ 고객감사 금리인하 이벤트
제목 : 삼성생명, 삼성전자 배당 확대 수혜…목표가↑-유안타
제목 : 11월 매크로환경 증시에 우호적…"저평가株 관심 가져야"
제목 : `IT주도주`만 날진 않는다..주변주로 온기 확산
제목 : 美 3분기 임금상승세 가팔라져…12월 금리인상 힘받나
제목 : [뉴욕증시 마감] 차기 연준 의장 지명 앞두고 강세
제목 : 뛸 채비하는 금리에…은행ㆍ보험사 '방긋' 여전사 '울상'
제목 : 금리 더 오른다…돈, 짧게 굴릴 '파킹족' 오라이~
제목 : [미리보는 웰스투어]부·울·경 부동산 투자할만한 곳은?…재테크 '한수' 얻을...
제목 : 국제유가, 감산연장 기대 상승지속…WTI 0.4%↑


# 채권 크롤링

In [3]:
from bs4 import BeautifulSoup, Comment
from urllib.request import urlopen
import pandas as pd
from selenium import webdriver
import time
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests

In [6]:
lili = [3,4,5,6,7,11,12,13,14,15,19,20,21,22,23,27,28,29,30,31,35,36,37,38,39,43,44,45,46,47]
for i in range(1,105) :
    url = "https://finance.naver.com/research/debenture_list.nhn?keyword=&brokerCode=&searchType=writeDate&writeFromDate=2005-01-01&writeToDate=2017-12-31&x=0&y=0&page="+ str(i)
    driver = webdriver.Chrome("chromedriver.exe")   
    driver.get(url)    

    idx = 0;
    
    
    for k in lili : 
        a= driver.find_element_by_xpath("""//*[@id="contentarea_left"]/div[3]/table[1]/tbody/tr["""+str(k)+"""]/td[3]/a""")
        b= driver.find_element_by_xpath("""//*[@id="contentarea_left"]/div[3]/table[1]/tbody/tr["""+str(k)+"""]/td[2]""")
        c= driver.find_element_by_xpath("""//*[@id="contentarea_left"]/div[3]/table[1]/tbody/tr["""+str(k)+"""]/td[4]""") 
        
        time.sleep(2)
        
        pdf_link = a.get_attribute('href')
        bb = b.text
        cc = c.text
        file = requests.get(pdf_link)
        filename = str(cc) + str(bb) + str(idx)+'.pdf'
        
        print(filename)
        
        with open(filename, 'wb') as f:
            f.write(file.content)
        idx +=1

17.12.27하나금융투자0.pdf
17.12.26현대차증권1.pdf
17.12.20하나금융투자2.pdf
17.12.20현대차증권3.pdf
17.12.18현대차증권4.pdf
17.12.15교보증권5.pdf
17.12.15하나금융투자6.pdf
17.12.14미래에셋대우7.pdf
17.12.14이베스트증권8.pdf
17.12.13하나금융투자9.pdf
17.12.08현대차증권10.pdf
17.12.07하나금융투자11.pdf
17.12.06미래에셋대우12.pdf
17.12.04현대차증권13.pdf
17.12.04현대차증권14.pdf
17.12.01이베스트증권15.pdf
17.12.01미래에셋대우16.pdf
17.12.01미래에셋대우17.pdf
17.12.01현대차증권18.pdf
17.11.30교보증권19.pdf
17.11.30이베스트증권20.pdf
17.11.29교보증권21.pdf
17.11.29하나금융투자22.pdf
17.11.28교보증권23.pdf
17.11.27교보증권24.pdf
17.11.27현대차증권25.pdf
17.11.23교보증권26.pdf
17.11.22미래에셋대우27.pdf
17.11.22미래에셋대우28.pdf
17.11.22교보증권29.pdf
17.11.21교보증권0.pdf
17.11.21교보증권1.pdf
17.11.20교보증권2.pdf
17.11.20IBK투자증권3.pdf
17.11.17미래에셋대우4.pdf
17.11.17교보증권5.pdf
17.11.17하나금융투자6.pdf
17.11.17하나금융투자7.pdf
17.11.16교보증권8.pdf
17.11.15교보증권9.pdf
17.11.15하나금융투자10.pdf
17.11.14미래에셋대우11.pdf
17.11.14교보증권12.pdf
17.11.14하나금융투자13.pdf
17.11.14하나금융투자14.pdf
17.11.14하나금융투자15.pdf
17.11.14현대차증권16.pdf
17.11.13키움증권17.pdf
17.11.13교보증권18.pdf
17.11.10미래에셋대우19.pdf
17.11.10교보